# Word-level language modeling using PyTorch

[Reference Source: PyTorch Example from SageMaker](https://github.com/awslabs/amazon-sagemaker-examples/tree/master/sagemaker-python-sdk/pytorch_lstm_word_language_model)

## Contents

1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Train)
1. [Host](#Host)

---

## Setup

_This notebook was created and tested on an ml.p2.xlarge notebook instance._

Let's start by creating a SageMaker session and specifying:

- The S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See [the documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html) for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the sagemaker.get_execution_role() with appropriate full IAM role arn string(s).


In [1]:
import sagemaker

sagemaker_session = sagemaker.Session()

'''
A session stores configuration state and allows you to create service clients and resources.
sagemaker.session.Session - AWS service calls are delegated to an underlying Boto3 session, 
which by default is initialized using the AWS configuration chain. 
When you make an Amazon SageMaker API call that accesses an S3 bucket location and one is not specified, 
the Session creates a default bucket based on a naming convention which includes the current AWS account ID.
'''

bucket = sagemaker_session.default_bucket()

'''
Form of the name of the bucket - sagemaker-{region}-{AWS account ID} Return the name of the default bucket to use in relevant Amazon SageMaker interactions.

'''

prefix = 'sagemaker/DEMO-pytorch-rnn-lstm'

'''
Used later
'''

role = sagemaker.get_execution_role()
'''
Get the execution role for the notebook instance. This is the IAM role that you created when you created your notebook instance. You pass the role to the tuning job.
'''

## Data
### Getting the data
As mentioned above we are going to use [the wikitext-2 raw data](https://www.salesforce.com/products/einstein/ai-research/the-wikitext-dependency-language-modeling-dataset/). This data is from Wikipedia and is licensed CC-BY-SA-3.0. Before you use this data for any other purpose than this example, you should understand the data license, described at https://creativecommons.org/licenses/by-sa/3.0/

This dataset is provided by SalesForce, The WikiText language modeling dataset is a collection of over 100 million tokens extracted from the set of verified Good and Featured articles on Wikipedia. What we have here is a good example of how English language flows.

### Examples
= Gold dollar =

 The gold dollar or gold one @-@ dollar piece was a coin struck as a regular issue by the United States Bureau of the Mint from 1849 to 1889 . The coin had three types over its lifetime , all designed by Mint Chief Engraver James B. Longacre . The Type 1 issue had the smallest diameter of any United States coin ever minted .
 A gold dollar had been proposed several times in the 1830s and 1840s , but was not initially adopted . Congress was finally galvanized into action by the increased supply of bullion caused by the California gold rush , and in 1849 authorized a gold dollar . In its early years , silver coins were being hoarded or exported , and the gold dollar found a ready place in commerce . Silver again circulated after Congress in 1853 required that new coins of that metal be made lighter , and the gold dollar became a rarity in commerce even before federal coins vanished from circulation because of the economic disruption caused by the American Civil War .
 
 = Super Mario Land =

 Super Mario Land is a 1989 side @-@ scrolling platform video game , the first in the Super Mario Land series , developed and published by Nintendo as a launch title for their Game Boy handheld game console . In gameplay similar to that of the 1985 Super Mario Bros. , but resized for the smaller device 's screen , the player advances Mario to the end of 12 levels by moving to the right and jumping across platforms to avoid enemies and pitfalls . Unlike other Mario games , Super Mario Land is set in Sarasaland , a new environment depicted in line art , and Mario pursues Princess Daisy . The game introduces two Gradius @-@ style shooter levels .
 At Nintendo CEO Hiroshi Yamauchi 's request , Game Boy creator Gunpei Yokoi 's Nintendo R & D1 developed a Mario game to sell the new console . It was the first portable version of Mario and the first to be made without Mario creator and Yokoi protégé Shigeru Miyamoto . Accordingly , the development team shrunk Mario gameplay elements for the device and used some elements inconsistently from the series . Super Mario Land was expected to showcase the console until Nintendo of America bundled Tetris with new Game Boys . The game launched alongside the Game Boy first in Japan ( April 1989 ) and later worldwide . Super Mario Land was later rereleased for the Nintendo 3DS via Virtual Console in 2011 again as a launch title , which featured some tweaks to the game 's presentation .
 Initial reviews were laudatory . Reviewers were satisfied with the smaller Super Mario Bros. , but noted its short length . They considered it among the best of the Game Boy launch titles . The handheld console became an immediate success and Super Mario Land ultimately sold over 18 million copies , more than that of Super Mario Bros. 3 . Both contemporaneous and retrospective reviewers praised the game 's soundtrack . Later reviews were critical of the compromises made in development and noted Super Mario Land 's deviance from series norms . The game begot a series of sequels , including the 1992 Super Mario Land 2 : 6 Golden Coins , 1994 Wario Land : Super Mario Land 3 , and 2011 Super Mario 3D Land , though many of the original 's mechanics were not revisited . The game was included in several top Game Boy game lists and debuted Princess Daisy as a recurring Mario series character .
 
= = = Sinclair Scientific Programmable = = =

 The Sinclair Scientific Programmable was introduced in 1975 , with the same case as the Sinclair Oxford . It was larger than the Scientific , at 73 by 155 by 34 millimetres ( 2 @.@ 9 in × 6 @.@ 1 in × 1 @.@ 3 in ) , and used a larger  battery , but could also be powered by mains electricity .
 It had 24 @-@ step programming abilities , which meant it was highly limited for many purposes . It also lacked functions for the natural logarithm and exponential function . Constants used in programs were required to be integers , and the programming was wasteful , with start and end quotes needed to use a constant in a program .
 However , included with the calculator was a library of over 120 programs that that performed common operations in mathematics , geometry , statistics , finance , physics , electronics , engineering , as well as fluid mechanics and materials science . The full library of standard programs contained over 400 programs in the Sinclair Program Library .

### Dataset statistics
In comparison to the Mikolov processed version of the Penn Treebank (PTB), the WikiText datasets are larger. WikiText-2 aims to be of a similar size to the PTB while WikiText-103 contains all articles extracted from Wikipedia. The WikiText datasets also retain numbers (as opposed to replacing them with N), case (as opposed to all text being lowercased), and punctuation (as opposed to stripping them out).

![Dataset statistics](../img/dataset-statistics.png)

In [2]:
%%bash
wget http://research.metamind.io.s3.amazonaws.com/wikitext/wikitext-2-raw-v1.zip
unzip -n wikitext-2-raw-v1.zip
cd wikitext-2-raw
mv wiki.test.raw test && mv wiki.train.raw train && mv wiki.valid.raw valid
# Moving the pre-divided datasets into Test, Train and Validation directories.

Archive:  wikitext-2-raw-v1.zip
   creating: wikitext-2-raw/
  inflating: wikitext-2-raw/wiki.test.raw  
  inflating: wikitext-2-raw/wiki.valid.raw  
  inflating: wikitext-2-raw/wiki.train.raw  


--2019-11-26 19:25:06--  http://research.metamind.io.s3.amazonaws.com/wikitext/wikitext-2-raw-v1.zip
Resolving research.metamind.io.s3.amazonaws.com (research.metamind.io.s3.amazonaws.com)... 52.216.98.11
Connecting to research.metamind.io.s3.amazonaws.com (research.metamind.io.s3.amazonaws.com)|52.216.98.11|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4721645 (4.5M) [application/zip]
Saving to: ‘wikitext-2-raw-v1.zip’

     0K .......... .......... .......... .......... ..........  1%  359K 13s
    50K .......... .......... .......... .......... ..........  2% 2.21M 7s
   100K .......... .......... .......... .......... ..........  3% 1.01M 6s
   150K .......... .......... .......... .......... ..........  4% 71.1M 5s
   200K .......... .......... .......... .......... ..........  5%  728K 5s
   250K .......... .......... .......... .......... ..........  6% 82.5M 4s
   300K .......... .......... .......... .......... ..........  7% 70.2M 3s
   350K .......

Let's preview what data looks like.

In [3]:
!head -5 wikitext-2-raw/train
#Lets see how the train dataset looks like

 
 = Valkyria Chronicles III = 
 
 Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . 
 The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more forgiving for series 

### Uploading the data to S3
We are going to use the `sagemaker.Session.upload_data` function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use later when we start the training job.



In [20]:
inputs = sagemaker_session.upload_data(path='wikitext-2-raw', bucket=bucket, key_prefix=prefix)

'''
S3 object key name prefix (default: ‘data’). S3 uses the prefix to create a directory structure for the bucket content that it display in the S3 console.

Tree of the datasets - 

├── wikitext-2-raw
│   ├── test
│   ├── train
│   └── valid
'''

print('input spec (in this case, just an S3 path): {}'.format(inputs))

input spec (in this case, just an S3 path): s3://sagemaker-us-west-2-111652037296/sagemaker/DEMO-pytorch-rnn-lstm-2


## Train
### Training script
We need to provide a training script that can run on the SageMaker platform. The training script is very similar to a training script you might run outside of SageMaker, but you can access useful properties about the training environment through various environment variables, such as:

* `SM_MODEL_DIR`: A string representing the path to the directory to write model artifacts to.
  These artifacts are uploaded to S3 for model hosting.
* `SM_OUTPUT_DATA_DIR`: A string representing the filesystem path to write output artifacts to. Output artifacts may
  include checkpoints, graphs, and other files to save, not including model artifacts. These artifacts are compressed
  and uploaded to S3 to the same S3 prefix as the model artifacts.

Supposing one input channel, 'training', was used in the call to the PyTorch estimator's `fit()` method,
the following will be set, following the format `SM_CHANNEL_[channel_name]`:

* `SM_CHANNEL_TRAINING`: A string representing the path to the directory containing data in the 'training' channel.

The script that we will use in this example is stored in GitHub repo 
[https://github.com/awslabs/amazon-sagemaker-examples/tree/training-scripts](https://github.com/awslabs/amazon-sagemaker-examples/tree/training-scripts), 
under the branch `training-scripts`. It is a public repo so we don't need authentication to access it. Let's specify the `git_config` argument here: 


A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to `model_dir` so that it can be hosted later. Hyperparameters are passed to your script as arguments and can be retrieved with an `argparse.ArgumentParser` instance. 

For example, the script run by this notebook: 
[https://github.com/awslabs/amazon-sagemaker-examples/blob/training-scripts/pytorch-rnn-scripts/train.py](https://github.com/awslabs/amazon-sagemaker-examples/blob/training-scripts/pytorch-rnn-scripts/train.py). 

For more information about training environment variables, please visit [SageMaker Containers](https://github.com/aws/sagemaker-containers).

In the current example we also need to provide source directory, because training script imports data and model classes from other modules. The source directory is 
[https://github.com/awslabs/amazon-sagemaker-examples/blob/training-scripts/pytorch-rnn-scripts/](https://github.com/awslabs/amazon-sagemaker-examples/blob/training-scripts/pytorch-rnn-scripts/). We should provide 'pytorch-rnn-scripts' for `source_dir` when creating the Estimator object, which is a relative path inside the Git repository. 


Lets see the training script in details - this training script is located here - 

```bash
├── pytorch-rnn-scripts
│   ├── data.py
│   ├── generate.py
│   ├── __init__.py
│   ├── rnn.py
│   └── train.py
```

```python
import data
```

Here we import data.py, data.py has functions for tokenizing and creating a corpus for consumptions. A few relevant details here - [tokens](https://github.com/nicolas-ivanov/tf_seq2seq_chatbot/issues/15#issuecomment-246106807)

Then we have hyperparamters being passed to this script, you can see this in the training blob


```python
# Hyperparameters sent by the client are passed as command-line arguments to the script.
parser.add_argument('--emsize', type=int, default=200,
                    help='size of word embeddings')
parser.add_argument('--nhid', type=int, default=200,
                    help='number of hidden units per layer')
parser.add_argument('--nlayers', type=int, default=2,
                    help='number of layers')
parser.add_argument('--lr', type=float, default=20,
                    help='initial learning rate')
parser.add_argument('--clip', type=float, default=0.25,
                    help='gradient clipping')
parser.add_argument('--epochs', type=int, default=40,
                    help='upper epoch limit')
parser.add_argument('--batch_size', type=int, default=20, metavar='N',
                    help='batch size')
parser.add_argument('--bptt', type=int, default=35,
                    help='sequence length')
parser.add_argument('--dropout', type=float, default=0.2,
                    help='dropout applied to layers (0 = no dropout)')
parser.add_argument('--tied', type=bool, default=False,
                    help='tie the word embedding and softmax weights')
parser.add_argument('--seed', type=int, default=1111,
                    help='random seed')
parser.add_argument('--log-interval', type=int, default=200, metavar='N',
                    help='report interval')
```
Then we have details of file paths - 

```python
# Data and model checkpoints/otput directories from the container environment
parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR'])
parser.add_argument('--output-data-dir', type=str, default=os.environ['SM_OUTPUT_DATA_DIR'])
parser.add_argument('--data-dir', type=str, default=os.environ['SM_CHANNEL_TRAINING'])
```

Here are some logs from when a job like this was run - 

```json
SM_TRAINING_ENV=
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "training": "/opt/ml/input/data/training"
    },
    "current_host": "algo-1",
    "framework_module": "sagemaker_pytorch_container.training:main",
    "hosts": [
        "algo-1"
    ],
    "hyperparameters": {
        "epochs": 6,
        "tied": true
    },
    "input_config_dir": "/opt/ml/input/config",
    "input_data_config": {
        "training": {
            "RecordWrapperType": "None",
            "S3DistributionType": "FullyReplicated",
            "TrainingInputMode": "File"
        }
    },
    "input_dir": "/opt/ml/input",
    "is_master": true,
    "job_name": "sagemaker-pytorch-2019-11-26-19-32-08-962",
    "log_level": 20,
    "master_hostname": "algo-1",
    "model_dir": "/opt/ml/model",
    "module_dir": "s3://sagemaker-us-west-2-111652037296/sagemaker-pytorch-2019-11-26-19-32-08-962/source/sourcedir.tar.gz",
    "module_name": "train",
    "network_interface_name": "eth0",
    "num_cpus": 4,
    "num_gpus": 1,
    "output_data_dir": "/opt/ml/output/data",
    "output_dir": "/opt/ml/output",
    "output_intermediate_dir": "/opt/ml/output/intermediate",
    "resource_config": {
        "current_host": "algo-1",
        "hosts": [
            "algo-1"
        ],
        "network_interface_name": "eth0"
    },
    "user_entry_point": "train.py"
}
```

Here are some environment variables - 

```json
SM_USER_ARGS=["--epochs","6","--tied","True"]
SM_OUTPUT_INTERMEDIATE_DIR=/opt/ml/output/intermediate
SM_CHANNEL_TRAINING=/opt/ml/input/data/training
SM_HP_TIED=true
SM_HP_EPOCHS=6
PYTHONPATH=/usr/local/bin:/usr/lib/python36.zip:/usr/lib/python3.6:/usr/lib/python3.6/lib-dynload:/usr/local/lib/python3.6/dist-packages:/usr/lib/python3/dist-packages
Invoking script with the following command:
/usr/bin/python -m train --epochs 6 --tied True

Namespace(batch_size=20, bptt=35, clip=0.25, data_dir='/opt/ml/input/data/training', dropout=0.2, emsize=200, epochs=6, log_interval=200, lr=20, model_dir='/opt/ml/model', nhid=200, nlayers=2, output_data_dir='/opt/ml/output/data', seed=1111, tied=True)
```

You can find the logs by going to the training jobs in the Amazon SageMaker Dashboard

```python
# Set the random seed manually for reproducibility.
torch.manual_seed(args.seed)
```

This seed you can seed was 1111 in the above example. You can use torch.manual_seed() to seed the RNG for all devices (both CPU and CUDA). Completely reproducible results are not guaranteed across PyTorch releases, individual commits or different platforms. Furthermore, results need not be reproducible between CPU and GPU executions, even when using identical seeds. However, in order to make computations deterministic on your specific problem on one specific platform and PyTorch release, there are a couple of steps to take. This is one of these steps. More details [here(https://pytorch.org/docs/stable/notes/randomness.html)







### Run training in SageMaker
The PyTorch class allows us to run our training function as a training job on SageMaker infrastructure. We need to configure it with our training script and source directory, an IAM role, the number of training instances, and the training instance type. In this case we will run our training job on ```ml.p2.xlarge``` instance. As you can see in this example you can also specify hyperparameters. 

Here we are using a prebuilt container for training our script, if you want to create your own please navigate to - https://github.com/aws/sagemaker-pytorch-container


In [7]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='train.py',
                    role=role,
                    framework_version='1.1.0',
                    train_instance_count=1,
                    train_instance_type='ml.p2.xlarge',
                    source_dir='pytorch-rnn-scripts',
                    # available hyperparameters: emsize, nhid, nlayers, lr, clip, epochs, batch_size,
                    #                            bptt, dropout, tied, seed, log_interval
                    hyperparameters={
                        'epochs': 6,
                        'tied': True
                    })


After we've constructed our PyTorch object, we can fit it using the data we uploaded to S3. SageMaker makes sure our data is available in the local filesystem, so our training script can simply read the data from disk.

In [8]:
estimator.fit({'training': inputs})

2019-11-26 19:32:10 Starting - Starting the training job...
2019-11-26 19:32:11 Starting - Launching requested ML instances......
2019-11-26 19:33:11 Starting - Preparing the instances for training.........
2019-11-26 19:35:04 Downloading - Downloading input data
2019-11-26 19:35:04 Training - Downloading the training image...
2019-11-26 19:35:34 Training - Training image download completed. Training in progress..
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2019-11-26 19:35:36,155 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2019-11-26 19:35:36,180 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2019-11-26 19:35:36,790 sagemaker_pytorch_container.training INFO     Invoking user training script.
2019-11-26 19:35:37,053 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2019-11-26 19:35:37,053 sagema

## Host
### Hosting script
We are going to provide custom implementation of `model_fn`, `input_fn`, `output_fn` and `predict_fn` hosting functions in a separate file, which is in the same Git repo as the training script: 
[https://github.com/awslabs/amazon-sagemaker-examples/blob/training-scripts/pytorch-rnn-scripts/generate.py](https://github.com/awslabs/amazon-sagemaker-examples/blob/training-scripts/pytorch-rnn-scripts/generate.py). 
We will use Git integration for hosting too since the hosting code is also in the Git repo. 


You can also put your training and hosting code in the same file but you would need to add a main guard (`if __name__=='__main__':`) for the training code, so that the container does not inadvertently run it at the wrong point in execution during hosting.

### Import model into SageMaker
The PyTorch model uses a npy serializer and deserializer by default. For this example, since we have a custom implementation of all the hosting functions and plan on using JSON instead, we need a predictor that can serialize and deserialize JSON.

In [9]:
from sagemaker.predictor import RealTimePredictor, json_serializer, json_deserializer

class JSONPredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(JSONPredictor, self).__init__(endpoint_name, sagemaker_session, json_serializer, json_deserializer)

Since hosting functions implemented outside of train script we can't just use estimator object to deploy the model. Instead we need to create a PyTorchModel object using the latest training job to get the S3 location of the trained model data. Besides model data location in S3, we also need to configure PyTorchModel with the script and source directory (because our `generate` script requires model and data classes from source directory), an IAM role.

In [10]:
from sagemaker.pytorch import PyTorchModel

training_job_name = estimator.latest_training_job.name
desc = sagemaker_session.sagemaker_client.describe_training_job(TrainingJobName=training_job_name)
trained_model_location = desc['ModelArtifacts']['S3ModelArtifacts']
model = PyTorchModel(model_data=trained_model_location,
                     role=role,
                     framework_version='1.0.0',
                     entry_point='generate.py',
                     source_dir='pytorch-rnn-scripts',
                     predictor_cls=JSONPredictor)

### Create endpoint

Now the model is ready to be deployed at a SageMaker endpoint and we are going to use the `sagemaker.pytorch.model.PyTorchModel.deploy` method to do this. We can use a CPU-based instance for inference (in this case an ml.m4.xlarge), even though we trained on GPU instances, because at the end of training we moved model to cpu before returning it. This way we can load trained model on any device and then move to GPU if CUDA is available. 


In [11]:
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

--------------------------------------------------------------------------------------!

### Evaluate
We are going to use our deployed model to generate text by providing random seed, temperature (higher will increase diversity) and number of words we would like to get.

In [12]:
input = {
    'seed': 200,
    'temperature': 510.0,
    'words': 200
}
response = predictor.predict(input)
print(response)

422nd wetland Potro integrating Eighty Demand lanyard sanctified Cévennes agendas detonations apology
Escorial DeMille Charlynch helix Zeffirelli Victorien Hessian PRG imitators sheath barbershops Suresnes
classifications roaming Alford RPGs guts Marshall Godrich Briefing Branislav 560 Roscoe souls
beheaded Borough AIL Shoko Cowper Potworowski goalkeepers Kenith Roumette Ortona Ismailia S.259
Wreath partied Bag survived applications annealed fisherman Plains luncheon Đình Tokyo heckles
egos Dobson Aag HevyDevy Investigates able Hoodoo Hendrie quatre Needled Prussian visitor
Hashomer Redfearn Matteino Stjepan Yiwu Bree roof Surdas simpler Measure bey 411
Babang widened Altrock Kulukundis Maverick Wing startling Kostrzewska Featley emails yes Odetta
Venture grammars Nachtigall Franka silence Kommunistblad 837 emcee 562 hOWLetts deducted reproductive
traveller difficult Rogatti 90s Casino Tracey Screen sabotaging Udell abdicated anglica College
lengthy Yuji Nonsense Forth realistic Ioke Q

### Cleanup

After you have finished with this example, remember to delete the prediction endpoint to release the instance(s) associated with it.


In [13]:
sagemaker_session.delete_endpoint(predictor.endpoint)